# Actividad en clase 5: Simulación del VaR por Montecarlo 
#### Luis Eduardo Jiménez del Muro
#### 26/02/2025
---

In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import random
import itertools

# 1. Carga las acciones del SP 500 con su sector y sus tickers

## Tickers del S&P500

In [2]:
sp = pd.read_excel('sp_500.xlsx')[['Symbol', 'GICS_Sector']]
sp.head()

,Symbol,GICS_Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology


## Descargar los precios del S&P500

In [3]:
closes_sp = yf.download(sp['Symbol'].tolist(), start='2022-01-01', end='2025-01-01')['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  503 of 503 completed

16 Failed downloads:
['FRC', 'CDAY', 'ATVI', 'DISH', 'RE', 'PKI', 'PEAK', 'FISV', 'BRK.B', 'ABC']: YFTzMissingError('possibly delisted; no timezone found')
['WRK', 'PXD', 'CTLT', 'MRO', 'FLT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2025-01-01) (Yahoo error = "No data found, symbol may be delisted")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2025-01-01)')


## Eliminar las acciones que no se descargaron correctamente

In [4]:
closes_sp = closes_sp.dropna(axis=1)
closes_sp.head()

Ticker,A,AAL,AAP,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,153.272263,18.750000,218.612747,178.879913,119.733459,130.754242,42.362530,389.190247,564.369995,167.763382,...,85.903885,61.547112,56.641758,53.827454,112.315765,128.309464,122.322250,583.900024,56.988934,227.280441
2022-01-04,148.090759,19.020000,218.862000,176.609665,119.503563,127.679108,42.914051,386.408997,554.000000,166.248184,...,85.629845,61.936642,58.772293,54.101856,113.507248,128.995499,123.563179,587.599976,58.612373,218.627396
2022-01-05,145.553818,18.680000,218.308014,171.911865,120.131310,127.105431,42.410072,379.604095,514.429993,163.681747,...,82.458733,62.543602,59.503269,53.231373,111.960220,127.360252,123.155861,558.179993,58.745441,210.314270
2022-01-06,146.063156,18.570000,223.109055,169.042053,119.565468,127.086662,42.657307,361.272888,514.119995,164.202621,...,82.840439,62.054420,60.902817,53.193535,111.806496,128.675964,122.132797,555.159973,61.158432,211.149490
2022-01-07,142.174530,19.280001,219.822220,169.209137,119.255989,127.481598,42.856995,354.343658,510.700012,159.893723,...,82.752350,62.597939,61.402016,53.363838,110.912895,127.200531,121.393906,530.859985,62.178608,205.002014


## Obtener el sector de cada acción que si se descargó

In [5]:
tickers_sp = closes_sp.columns.to_list()
tickers_sector = sp[sp["Symbol"].isin(tickers_sp)]
tickers_sector.head()

,Symbol,GICS_Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology


# 2. Realiza 200 combinaciones de portafolios de 8 activos, cuidando no incluir más de 4 activos del mismo sector

In [6]:
n_acciones = 8
max_per_sector = 4

# Función que toma muestras de 8 acciones delimitando un máximo por sector
def sample_stocks(df, n_acciones, max_per_sector):
    sampled = []
    sector_counts = {}

    shuffled_df = df.sample(frac=1).reset_index(drop=True) 

    for _, row in shuffled_df.iterrows():
        sector = row["GICS_Sector"]
        if sector_counts.get(sector, 0) < max_per_sector:
            sampled.append(row)
            sector_counts[sector] = sector_counts.get(sector, 0) + 1
        if len(sampled) == n_acciones:
            break

    return pd.DataFrame(sampled)

n_port = 200

# Simula n portafolios 
samples = [sample_stocks(tickers_sector, n_acciones, max_per_sector) for _ in range(n_port)]
samples[0]

,Symbol,GICS_Sector
0,DVA,Health Care
1,AME,Industrials
2,OMC,Communication Services
3,ORLY,Consumer Discretionary
4,SCHW,Financials
5,GPC,Consumer Discretionary
6,TFX,Health Care
7,WELL,Real Estate


# 3. Para cada una de esas 200 combinaciones, simula 10K posibles ponderaciones de pesos y calcula el ratio de sharpe para cada ponderación.

## Tickers de cada portafolio

In [7]:
tickers_port = []
for i in samples:
    tickers_port.append(i['Symbol'].tolist())

## Precios de cada portafolio 

In [8]:
list_prices_ports = []
for i in tickers_port:
    list_prices_ports.append(closes_sp[i])

## Simulación de portafolios

In [17]:
all_portfolios = []

rf = 0.0415
num_portafolios = 10000

for i in range(len(tickers_port)):
    returns = list_prices_ports[i].pct_change().dropna()
    cov_matrix = returns.cov()

    annual_returns = returns.mean()*252
    annual_cov = cov_matrix * 252

    weights_array = np.zeros((num_portafolios, n_acciones))
    returns_array = np.zeros(num_portafolios)
    risk_array =  np.zeros(num_portafolios)
    sharpe_array = np.zeros(num_portafolios)

    for j in range(num_portafolios):
        weights = np.random.random(n_acciones)
        weights = weights / np.sum(weights)

        portfolio_return = np.dot(annual_returns, weights)
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))

        weights_array[j, :] = weights
        returns_array[j] = portfolio_return
        risk_array[j] = portfolio_risk
        sharpe_array[j] = (portfolio_return - rf) / portfolio_risk

    portfolios = pd.DataFrame({
        'Return':returns_array,
        'Risk': risk_array,
        'Sharpe':sharpe_array
    })

    for k, ticker in enumerate(tickers_port[i]):
        portfolios[ticker + ' Weight'] = weights_array[:, k]

    all_portfolios.append(portfolios)


# De los 2M de filas resultantes, estima los 10 portafolios que maximizaron el ratio de sharpe. Esos 10 deben ser de combinaciones de activos distintas (únicas). 

In [18]:
len(all_portfolios), all_portfolios[0].shape

(200, (10000, 11))

## Unir todos los dataframes en uno solo

In [19]:
df_all = pd.concat(all_portfolios)
df_all

,Return,Risk,Sharpe,DVA Weight,AME Weight,OMC Weight,ORLY Weight,SCHW Weight,GPC Weight,TFX Weight,...,BAC Weight,KR Weight,BLK Weight,ETR Weight,PNW Weight,AAL Weight,BA Weight,BRO Weight,JCI Weight,AMAT Weight
0,0.066298,0.173491,0.142933,0.138223,0.158463,0.049281,0.044860,0.106619,0.086635,0.201560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.065314,0.182491,0.130493,0.069431,0.101209,0.012386,0.080009,0.235100,0.233861,0.096439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061139,0.178971,0.109732,0.123998,0.162064,0.044749,0.007295,0.185221,0.099706,0.172405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.078926,0.175500,0.213254,0.022015,0.143443,0.124867,0.165217,0.179572,0.168700,0.093702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.088540,0.173272,0.271481,0.028866,0.239541,0.038230,0.237606,0.126588,0.197629,0.072559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.070436,0.175933,0.164474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,0.059500,0.172169,0.104551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,0.084599,0.202454,0.212882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,0.058479,0.168535,0.100743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Crear un indicador de los tickers para eliminar combinaciones repetidas

In [20]:
tickers_cols = [col for col in df_all.columns if "Weight" in col]
df_all['Unique_Combination'] = df_all[tickers_cols].apply(lambda row: "-".join(sorted(row.index[row > 0])), axis=1)

df_all = df_all.drop_duplicates(subset=["Unique_Combination"])
df_all = df_all.drop(columns='Unique_Combination')
df_all

C:\Users\epile\AppData\Local\Temp\ipykernel_17448\316282410.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all['Unique_Combination'] = df_all[tickers_cols].apply(lambda row: "-".join(sorted(row.index[row > 0])), axis=1)


,Return,Risk,Sharpe,DVA Weight,AME Weight,OMC Weight,ORLY Weight,SCHW Weight,GPC Weight,TFX Weight,...,BAC Weight,KR Weight,BLK Weight,ETR Weight,PNW Weight,AAL Weight,BA Weight,BRO Weight,JCI Weight,AMAT Weight
0,0.066298,0.173491,0.142933,0.138223,0.158463,0.049281,0.04486,0.106619,0.086635,0.20156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.113024,0.206597,0.346200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.086829,0.187047,0.242339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.087270,0.190568,0.240177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.017402,0.211417,-0.113985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.027497,0.214229,-0.065363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.062781,0.240088,0.088640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.082235,0.184936,0.220267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007586
0,0.118380,0.204160,0.376568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.072879,NaN,NaN,NaN,NaN,NaN,NaN


## Mejores 10 portafolios

In [21]:
top_10 = df_all.sort_values('Sharpe', ascending=False).head(10)
top_10.rename(columns=lambda col: col.replace(" Weight", "") if col.endswith(" Weight") else col, inplace=True)

best_portfolios = []

for _, row in top_10.iterrows():
    portfolio_df = row.to_frame().T
    portfolio_df = portfolio_df.dropna(axis=1)
    best_portfolios.append(portfolio_df)

## Simular 2 semanas de rendimientos para cada portafolio

### Función que simula rendimientos

In [22]:
def simular_rendimientos(mean_returns, cov_returns, prices, num_dias, stocks, weights):

    N = 10000
    portfolio_return = np.dot(weights, mean_returns)

    portfolio_simulated_returns = np.zeros((num_dias, N)) # filas x columnas
    mean_returns = np.full(shape = (num_dias, len(stocks)), fill_value=portfolio_return)

    for m in range(N):
        L = np.linalg.cholesky(cov_returns)
        Z = np.random.normal(size=(num_dias, len(stocks))) # Matriz de n_dias (filas) y n_stocks (columnas)
        daily_returns = mean_returns.T + np.dot(L, Z.T)
        portfolio_simulated_returns [:, m] = np.cumprod(np.dot(weights, daily_returns) + 1)
    
    return portfolio_simulated_returns

### Preparar las entradas de la función

In [23]:
best_portfolios_stocks = []
best_portfolios_weights = []
best_portfolios_prices = [] 
best_portfolios_returns = []
best_portfolios_cov = []

for i in range(len(best_portfolios)):
    best_portfolios_stocks.append(best_portfolios[i].columns[3:].tolist())
    best_portfolios_weights.append(best_portfolios[i].values[0][3:])
    best_portfolios_prices.append(closes_sp[best_portfolios[i].columns[3:].tolist()]) 
    best_portfolios_returns.append(closes_sp[best_portfolios[i].columns[3:].tolist()].pct_change().dropna().mean())
    best_portfolios_cov.append(closes_sp[best_portfolios[i].columns[3:].tolist()].pct_change().dropna().cov())

### Simular rendimientos

In [24]:
simulated_returns = []

n_dias = 14

for i in range(len(best_portfolios_prices)):
    simulated_returns.append(simular_rendimientos(best_portfolios_returns[i], best_portfolios_cov[i],
                                            best_portfolios_prices[i], n_dias,
                                            best_portfolios_stocks[i], best_portfolios_weights[i]))

### Tomar el VaR al 95% de confianza

In [25]:
var = []
nc = 95

for i in range(len(simulated_returns)):
    var.append(np.percentile(simulated_returns[i][-1,:] - 1, 100-nc))

### Agregar el VaR a los resultados de los 10 mejores portafolios

In [26]:
for i in range(len(best_portfolios)):
    best_portfolios[i]['VaR'] = var[i]

# 5. Para los 10 portafolios resultantes presenta:

+ El rendimiento anual esperado 
+ La volatilidad anual esperada 
+ El Ratio de Sharpe
+ El Value At Risk ( en términos porcentuales) a 2 semanas 
+ Los activos elegidos y su peso en el portafolio

In [27]:
from IPython.display import display_markdown

for i in range(len(best_portfolios)):
    display_markdown(f'### **Portafolio {i+1}**', raw=True)
    display(best_portfolios[i])

### **Portafolio 1**

,Return,Risk,Sharpe,ORCL,AVGO,FOX,DFS,MDLZ,TT,MGM,APA,VaR
0,0.266197,0.213947,1.05025,0.229862,0.203741,0.05762,0.148217,0.134329,0.196343,0.013317,0.016572,-0.06771


### **Portafolio 2**

,Return,Risk,Sharpe,WRB,HSY,TFC,PG,PAYX,IRM,TRGP,CTSH,VaR
0,0.204796,0.165936,0.984087,0.047826,0.098443,0.026003,0.218057,0.129957,0.233383,0.208789,0.037542,-0.05257


### **Portafolio 3**

...

,Return,Risk,Sharpe,ISRG,TMUS,WYNN,IFF,AXP,TDY,MAR,PM,VaR
0,0.175105,0.195544,0.683249,0.257518,0.22754,0.123077,0.045673,0.175287,0.041479,0.003847,0.125579,-0.06435


### **Portafolio 9**

,Return,Risk,Sharpe,AME,GLW,ODFL,CAH,CB,CAT,FFIV,SLB,VaR
0,0.165371,0.181873,0.681085,0.026531,0.12969,0.130528,0.139165,0.146029,0.156197,0.168045,0.103815,-0.062064


### **Portafolio 10**

,Return,Risk,Sharpe,MA,LDOS,ABT,AVGO,CI,HII,LOW,COP,VaR
0,0.156793,0.178747,0.645004,0.165448,0.002607,0.226259,0.190289,0.171568,0.140837,0.077816,0.025175,-0.058682
